In [ ]:
import cv2
import numpy as np
import math
from PIL import Image

In [ ]:
global img_3
img_3 = np.zeros([720,1080,3],dtype=np.uint8)
img_3.fill(255)

In [ ]:
def drawRectangle(action, x, y, flags, *userdata):
  # Referencing global variables 
  global top_left_corner, bottom_right_corner, img_3
  # Mark the top left corner when left mouse button is pressed
  if action == cv2.EVENT_LBUTTONDOWN:
    top_left_corner = [(x,y)]
    # When left mouse button is released, mark bottom right corner
  if action == cv2.EVENT_LBUTTONUP:
    bottom_right_corner = [(x,y)]
    # Draw the rectangle
    cv2.rectangle(img_3, top_left_corner[0], bottom_right_corner[0], (0,0,0),thickness=2)
    cv2.imshow("3 Channel Window",img_3)

In [ ]:
def cal_move_rect(first_transform, bottom_transform, top_left_corner, bottom_right_corner):
    vector = [(bottom_transform[0][0] - first_transform[0][0],bottom_transform[0][1]- first_transform[0][1])]
    tx=vector[0][0]
    ty=vector[0][1]
    T = np.array([[1,0,tx],[0,1,ty],[0,0,1]])
    A = np.array([top_left_corner[0][0],top_left_corner[0][1]])
    B = np.array([bottom_right_corner[0][0],bottom_right_corner[0][1]])
    rectangle = np.array([A,B])
    one = np.ones((rectangle.shape[0], 1))
    rectangle_matrix_ready = np.concatenate((rectangle,one), axis = 1)
    copyOfMatrixReady = rectangle_matrix_ready.copy()
    for i in range(rectangle_matrix_ready.shape[0]):
        copyOfMatrixReady[i] = T.dot(rectangle_matrix_ready[i])
    new_top_left_corner = [(int(copyOfMatrixReady[0][0]), int(copyOfMatrixReady[0][1]))]
    top_left_corner = new_top_left_corner
    new_bottom_right_corner = [(int(copyOfMatrixReady[1][0]), int(copyOfMatrixReady[1][1]))]
    bottom_right_corner = new_bottom_right_corner
    return top_left_corner, bottom_right_corner

In [ ]:
def moveRectangle(action, x, y, flags, *userdata):
    global first_transform, bottom_transform
    global top_left_corner, bottom_right_corner, img_3
    if action == cv2.EVENT_LBUTTONDOWN:
        first_transform = [(x,y)]
        # When left mouse button is released, mark bottom right corner
    elif action == cv2.EVENT_LBUTTONUP:
        bottom_transform = [(x,y)]
        top_left_corner,bottom_right_corner = cal_move_rect(first_transform, bottom_transform, top_left_corner,bottom_right_corner)

        img_3 = np.zeros([720,1080,3],dtype=np.uint8)
        img_3.fill(255)
        # print('new_top_left_corner',top_left_corner)
        # print('new_bottom_right_corner',bottom_right_corner)
        img_3 = cv2.rectangle(img_3, top_left_corner[0], bottom_right_corner[0], (0,0,0),thickness=2)
        cv2.imshow("3 Channel Window",img_3)